In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
"""from PIL import Image, ImageOps
import os
import pickle
from sklearn.model_selection import train_test_split
nclasses = -1
train = []
train_labels = []
class_names = {}
for dirname, _, filenames in os.walk('../input/hotel-id-to-combat-human-trafficking-2022-fgvc9/train_images'):
    nclasses += 1
    class_names[nclasses] = dirname.split("/")[-1]
    for filename in filenames:
        train.append(np.asarray(ImageOps.Image.open(os.path.join(dirname, filename)).resize((224,224))))
        train_labels.append(nclasses)
dataset = tf.data.Dataset.from_tensor_slices(train)

x_train, x_val, y_train, y_val = train_test_split(train, train_labels, test_size=0.05)
train_dataset = tf.data.Dataset.from_tensor_slices((x_train, y_train))
# Shuffle and slice the dataset.
train_dataset = train_dataset.shuffle(buffer_size=1024).batch(64)

# Now we get a test dataset.
val_dataset = tf.data.Dataset.from_tensor_slices((x_val, y_val))
val_dataset = test_dataset.batch(64)


with open('x_train.pkl', 'wb') as f:
    pickle.dump(x_train, f)
with open('x_val.pkl', 'wb') as f:
    pickle.dump(x_val, f)
with open('y_train.pkl', 'wb') as f:
    pickle.dump(y_train, f)
with open('y_val.pkl', 'wb') as f:
    pickle.dump(y_val, f)"""

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import pickle
with open('../input/70x70hotel/x_train (1).pkl', 'rb') as f:
    x_train = pickle.load(f)
with open('../input/70x70hotel/x_val (1).pkl', 'rb') as f:
    x_val = pickle.load(f)
with open('../input/70x70hotel/y_train (1).pkl', 'rb') as f:
    y_train = pickle.load(f)
with open('../input/70x70hotel/y_val (1).pkl', 'rb') as f:
    y_val = pickle.load(f)

In [ ]:
import os
np.array(x_train).shape
nclasses = 3316

In [ ]:
conv_arch = ((1, 64), (1, 128), (2, 256), (2, 512), (2, 512))

In [ ]:
import tensorflow as tf
def vgg_block(num_convs, num_channels):
    blk = tf.keras.models.Sequential()
    for _ in range(num_convs):
        blk.add(tf.keras.layers.Conv2D(num_channels,kernel_size=3,
                                    padding='same',activation='relu'))
    blk.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2))
    return blk

In [ ]:
def vgg(conv_arch):
    net = tf.keras.models.Sequential()
    # The convulational part
    for (num_convs, num_channels) in conv_arch:
        net.add(vgg_block(num_convs, num_channels))
    # The fully-connected part
    net.add(tf.keras.models.Sequential([
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(4096, activation='relu'),
        tf.keras.layers.Dropout(0.5),
        tf.keras.layers.Dense(4096, activation='relu'),
        tf.keras.layers.Dropout(0.5),
        tf.keras.layers.Dense(nclasses+1)]))
    return net

net = vgg(conv_arch)

In [ ]:
BATCH_SIZE = 16
LOSS = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
OPTIMIZER = tf.keras.optimizers.Adam(
    learning_rate=0.0001,
    beta_1=0.9,
    beta_2=0.999,
    epsilon=1e-07,
    amsgrad=False,
    name='Adam')
EPOCHS = 50

In [ ]:
from PIL import Image, ImageOps
import os
import pickle
from sklearn.model_selection import train_test_split
model = vgg(conv_arch)
model.compile(optimizer=OPTIMIZER,
              loss= LOSS,
              metrics=[tf.keras.metrics.Precision()])
history = model.fit(train_dataset, epochs=EPOCHS, batch_size=BATCH_SIZE,
                    validation_data=val_dataset)

test = []
images = []
for dirname, _, filenames in os.walk('../input/hotel-id-to-combat-human-trafficking-2022-fgvc9/test_images'):
    for filename in filenames:
        images.append(filename)
        test.append(np.asarray(ImageOps.Image.open(os.path.join(dirname, filename)).resize((70,70))))
test = np.array(test, dtype=np.float32)
print(test.shape)
y_pred = model.predict(test)
y_pred = list(map(np.argmax, y_pred))
d = {'image_id': images, 'hotel_id': list(map(class_names.get, y_pred))}
df = pd.DataFrame(data=d)
df.to_csv("submission.csv", index=False)

In [ ]:
"""from sklearn.neighbors import KNeighborsClassifier
import os
from PIL import Image, ImageOps
k = 5
knc = KNeighborsClassifier(n_neighbors = k)
knc.fit(train, train_labels)
test = []
images = []
for dirname, _, filenames in os.walk('../input/hotel-id-to-combat-human-trafficking-2022-fgvc9/test_images'):
    for filename in filenames:
        images.append(filename)
        test.append(np.asarray(ImageOps.grayscale(Image.open(os.path.join(dirname, filename)).resize((40,40)))).flatten())
        
y_pred = knc.predict(test)
d = {'image_id': images, 'hotel_id': list(map(class_names.get, y_pred))}
df = pd.DataFrame(data=d)
df.to_csv("submission.csv", index=False)"""

In [ ]:
"""import tensorflow_datasets
ds_train, ds_test = tensorflow_datasets.load('cifar100', split=['train', 'test'], shuffle_files=True, as_supervised=True)
assert isinstance(ds_train, tf.data.Dataset)
assert isinstance(ds_test, tf.data.Dataset)"""